In [1]:
# Import the necessary libraries
import sys
import os
import pandas as pd
import warnings

# Add the src folder to the path
sys.path.append(os.path.abspath('..'))
warnings.filterwarnings("ignore")

# Import the classes
from src.data_tool.data_providers.connect_wrds import get_wrds_data, get_security_daily
from src.data_tool.data_providers.yahoo_finance import yahoo_finance
from src.data_tool.data_providers.ploygon import polygon
from src.data_tool.data_providers.financial_datasets import FinancialDatasets
from src.database_tool.connect_db import ConnectDB
from src.database_tool.create_table import TableCreator

# Initialize the classes
yf = yahoo_finance()
pl = polygon()
fd = FinancialDatasets()

db =ConnectDB()
engine = db.get_engine()


FINANCIAL_DATASETS_API_KEY is set
YAHOO_CONSUMER_KEY and YAHOO_CONSUMER_SECRET are set
YAHOO_APP_ID is set
POLYGON_API_KEY is set
DB_USER and DB_PASSWORD are set
DB_USER and DB_PASSWORD are set
DB_USER and DB_PASSWORD are set


In [2]:
def get_database_size(db_instance, database_name):
    """
    Retrieves the total size of the database.
    """
    sql_statement = f"""
    SELECT table_schema "database_name", 
           sum(data_length + index_length) / 1024 / 1024 "database_size_mb" 
    FROM information_schema.TABLES 
    WHERE table_schema = '{database_name}'
    GROUP BY table_schema;
    """
    result = db_instance.execute_sql(sql_statement)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

database_name = 'Elite_Traders' # Replace with your database name if different
database_size_df = get_database_size(db, database_name)
print(database_size_df)

   database_name database_size_mb
0  Elite_Traders       0.78125000


In [3]:
def get_table_sizes(db_instance):
    """
    Retrieves the size of each table in the database.
    """
    sql_statement = "SHOW TABLE STATUS"
    result = db_instance.execute_sql(sql_statement)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    size_df = df[['Name', 'Data_length', 'Index_length', 'Data_free']]
    size_df['Total_size'] = size_df['Data_length'] + size_df['Index_length']
    size_df['Data_length_MB'] = size_df['Data_length'] / 1024 / 1024
    size_df['Index_length_MB'] = size_df['Index_length'] / 1024 / 1024
    size_df['Data_free_MB'] = size_df['Data_free'] / 1024 / 1024
    size_df['Total_size_MB'] = size_df['Total_size'] / 1024 / 1024
    return size_df

table_sizes_df = get_table_sizes(db)
print(table_sizes_df)

                         Name  Data_length  Index_length  Data_free  \
0      capital_structure_debt        16384             0          0   
1    capital_structure_equity        16384             0          0   
2   capital_structure_summary        16384             0          0   
3           financial_metrics        16384             0          0   
4            financial_ratios        16384             0          0   
5         fundamentals_annual        16384             0          0   
6      fundamentals_quarterly        49152             0          0   
7                 identifiers        16384             0          0   
8               insider_trade       163840             0          0   
9                       price        16384             0          0   
10             security_daily       475136             0          0   

    Total_size  Data_length_MB  Index_length_MB  Data_free_MB  Total_size_MB  
0        16384        0.015625              0.0           0.0       

In [4]:
db.show_tables()

,Tables
0,capital_structure_debt
1,capital_structure_equity
2,capital_structure_summary
3,financial_metrics
4,financial_ratios
5,fundamentals_annual
6,fundamentals_quarterly
7,identifiers
8,insider_trade
9,price


In [5]:
tc = TableCreator()
tc.create_all_tables()

DB_USER and DB_PASSWORD are set
Price table already exists
Financial Metrics table already exists
Insider Trade table already exists
Company News table created successfully
All tables created successfully


In [6]:
db.show_tables()

,Tables
0,capital_structure_debt
1,capital_structure_equity
2,capital_structure_summary
3,company_news
4,financial_metrics
5,financial_ratios
6,fundamentals_annual
7,fundamentals_quarterly
8,identifiers
9,insider_trade


In [10]:
news = db.read_table('company_news')

In [11]:
news.tail()

,id,polygon_id,ticker,title,author,publisher,published_utc,article_url,tickers,description,keywords,insights
282,283,2a992adbd8a6dd72e83aa9bf28b5176da249a716fa2303...,NVDA,CoreWeave IPO: Is this Fast-Growing AI Stock a...,Jeremy Bowman,The Motley Fool,2025-04-02 15:48:46,https://www.fool.com/investing/2025/04/02/core...,"[""CRWV"", ""NVDA"", ""MSFT""]","CoreWeave, a cloud infrastructure platform pur...","[""CoreWeave"", ""AI"", ""IPO"", ""Nvidia"", ""Microsof...","[{""ticker"": ""CRWV"", ""sentiment"": ""neutral"", ""s..."
283,284,5144eaef31e6469a48882a4c387c51e3d0b49f6ff107ce...,NVDA,Why Some Quantum Computing Stocks Soared Last ...,Anders Bylund,The Motley Fool,2025-04-02 16:52:36,https://www.fool.com/investing/2025/04/02/some...,"[""QBTS"", ""QBTS.WS"", ""QUBT"", ""RGTI"", ""RGTIW"", ""...",Quantum computing stocks had a volatile month ...,"[""quantum computing"", ""stock performance"", ""D-...","[{""ticker"": ""QBTS"", ""sentiment"": ""positive"", ""..."
284,285,cfa5d6f1390722d6889932741d57a0ad27f5b512003cd8...,NVDA,Why Nvidia Stock Lost 13% in March,Jeremy Bowman,The Motley Fool,2025-04-02 18:34:50,https://www.fool.com/investing/2025/04/02/why-...,"[""NVDA""]",Nvidia's stock fell 13% in March due to macroe...,"[""Nvidia"", ""AI"", ""semiconductor"", ""macroeconom...","[{""ticker"": ""NVDA"", ""sentiment"": ""neutral"", ""s..."
285,286,44aa6413805e29973e0509ad35a8538a0bcd545f9221d6...,NVDA,Liberation Day Tariffs: Potential Impact on th...,Zain Vawda,Investing.com,2025-04-02 19:24:00,https://www.investing.com/analysis/liberation-...,"[""AAPL"", ""MSFT"", ""NVDA""]",The article discusses the potential impact of ...,"[""tariffs"", ""trade war"", ""stock market"", ""S&P ...","[{""ticker"": ""AAPL"", ""sentiment"": ""neutral"", ""s..."
286,287,9f9c58226349b93485c4f8a796b03d103d74f74336db64...,NVDA,Markets Reel As Trump's Tariffs Hit — Magnific...,Piero Cingari,Benzinga,2025-04-02 21:35:44,https://www.benzinga.com/government/regulation...,"[""AAPL"", ""AMZN"", ""NVDA"", ""XLK"", ""XLY""]",President Trump's new tariffs triggered a shar...,"[""tariffs"", ""trade war"", ""tech stocks"", ""marke...","[{""ticker"": ""AAPL"", ""sentiment"": ""negative"", ""..."


In [ ]:
#db.drop_table('company_news')